In [1]:
import openpyxl
from openpyxl.styles import Alignment
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import os
import json
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time


In [2]:
cntyCds = {"CH","BD","CA","IN","KH","PH","SG","TH","AU","CL","TW","JP","CN","GB","EU","US"}

In [3]:
reffNoNm_lists = []

In [ ]:
import requests
import pandas as pd

# URL của API
url = "https://unipass.customs.go.kr/clip/prlstclsfsrch/retrievePrlstClsfCaseLst.do"

# Headers của yêu cầu
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7,fr-FR;q=0.6,fr;q=0.5',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://unipass.customs.go.kr',
    'Referer': 'https://unipass.customs.go.kr/clip/index.do',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

# Danh sách cntyCd và nơi lưu các trang đã quét
pages_info = {}

# Lần đầu tiên, ghi nhận startPage và totalPage
for cntyCd in cntyCds:
    pageIndex = 1
    # Thực hiện yêu cầu POST đầu tiên để lấy tổng số trang
    data = {
        'pageIndex': pageIndex,
        'pageUnit': 1000,
        'orderColumns': 'ENFR_DT desc',
        'cntyCd': cntyCd,
        'srchYn': 'Y',
        'scrnTp': 'WDTH',
        'lngaTp': 'org',
        'srchDtrmHsSgn':'+' 
    }
    response = requests.post(url, headers=headers, data=data, timeout=60).json()
    
    totalPageCount = response["paginationInfo"]["totalPageCount"]
    pages_info[cntyCd] = {'startPage': 1, 'totalPage': totalPageCount}

    print(f"Đã ghi nhận {cntyCd}: {totalPageCount} trang.")

# Lần tiếp theo, duyệt từng trang từ startPage đến totalPage
for cntyCd, page_data in pages_info.items():
    for pageIndex in range(page_data['startPage'], page_data['totalPage'] + 1):
        data = {
            'pageIndex': pageIndex,
            'pageUnit': 1000,
            'orderColumns': 'ENFR_DT desc',
            'cntyCd': cntyCd,
            'srchYn': 'Y',
            'scrnTp': 'WDTH',
            'lngaTp': 'org',
            'srchDtrmHsSgn':'+' 
        }
        totalPageCount = response["paginationInfo"]["totalPageCount"]
        response = requests.post(url, headers=headers, data=data, timeout=60).json()
        print(f"Đang tải {cntyCd}, trang {pageIndex}/{totalPageCount}. Số lượng: {response['uls_over']['count']} ,firstRecordIndex: {response["paginationInfo"]["firstRecordIndex"]}, lastRecordIndex: {response["paginationInfo"]["lastRecordIndex"]}")

        for item in response['uls_over']["itemList"]:
            item_info = {
                'area': cntyCd,
                'page': pageIndex,
                'cntyCd': item['CNTY_CD'],
                'baseYy': item['BASE_YY'],
                'reffNoNm': item['REFF_NO_NM_ORI'],
                'prlstClsfSrno': item['PRLST_CLSF_SRNO'],
            }
            print(item_info)
            reffNoNm_lists.append(item_info)



In [ ]:
# Tạo DataFrame từ danh sách
df = pd.DataFrame(reffNoNm_lists)
df.to_csv("all_link_list.csv",index = True)